### <span style="color: skyblue;">LCEL(LangChain Expression Language)</span>
* LangChain 프레임워크에서 모듈화된 워크플로우를 만들기 위해 제공되는 기능입니다.   
이를 통해 자연어 처리(NLP)와 AI 기반 애플리케이션에서 입력 데이터를 처리하고 원하는 출력을 생성하는 작업을 유연하고 재사용 가능하게 설계할 수 있습니다.

#### stuff와 map reduce의 차이

* <span style="color: skyblue;">stuff</span>는 질문을 query하여 관련있는 지문을 먼저 찾아서
찾은 지문 전체를 prompt로 넣고 llm을 돌린다.

* <span style="color: skyblue;">map reduce</span>는 질문을 기반으로 retriever를 통해 찾은 질문과 연관된 지문 단락을 단락 수 만큼 for 문 수행하면서 각 단락 기준으로 llm 의 결과를 저장함.
각각 저장된 llm의 결과를 prompt로 합쳐서 llm 이 최종 결과를 만들어 냄.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
# RunnableLambda: chain과 그 내부 어디에서든 function을 호출할 수 있도록 해준다.  
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

# OpenAI의 ChatGPT 모델을 초기화합니다. 'temperature'는 응답의 다양성을 조절합니다.
llm = ChatOpenAI(
    temperature=0.1,  # 낮은 값으로 설정하여 응답의 일관성을 높임
)

# 캐시 디렉토리를 로컬 파일 시스템에 정의합니다.
cache_dir = LocalFileStore("./.cache/")

# 텍스트를 청크로 나누기 위한 분할기를 정의합니다.
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",  # 텍스트를 줄바꿈 문자로 분할
    chunk_size=600,  # 각 청크의 최대 문자 수
    chunk_overlap=100,  # 청크 간 중복 문자 수
)

# 파일 로더를 사용하여 텍스트 파일("document.txt")을 로드합니다.
loader = UnstructuredFileLoader("./document.txt")

# 파일을 로드하고, 분할기를 사용하여 텍스트를 분할합니다.
docs = loader.load_and_split(text_splitter=splitter)

# OpenAI 임베딩을 초기화합니다.
embeddings = OpenAIEmbeddings()

# 캐시를 지원하는 임베딩을 생성합니다. 캐시는 저장소에 임베딩을 저장하고 재사용합니다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 분할된 문서를 기반으로 FAISS 벡터 스토어를 생성합니다.
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# 벡터 스토어를 검색 도구로 변환합니다.
retriever = vectorstore.as_retriever()

# **1단계 프롬프트 정의**
# 문서에서 질문과 관련된 텍스트를 추출하기 위한 프롬프트를 정의합니다.
map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """ 
            Use the following portion of a long document to see if any of the text is relevant to answer the question. 
            Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context} 
            """,
        ),
        ("human", "{question}"),  # 질문을 동적으로 삽입
    ]
)

# 1단계 체인을 생성하여 프롬프트와 언어 모델(LLM)을 연결합니다.
map_doc_chain = map_doc_prompt | llm

# **문서를 처리하는 사용자 정의 함수 정의**
def map_docs(inputs):
    documents = inputs["documents"]  # 검색된 문서를 가져옵니다.
    question = inputs["question"]  # 사용자 질문을 가져옵니다.
    return "\n\n".join(  # 각 문서에서 관련 텍스트를 추출하고 병합
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}  # 문서 내용과 질문 전달
        ).content
        for doc in documents
    )

# **문서 검색과 질문을 연결**
map_chain = {
    "documents": retriever,  # 문서를 검색합니다.
    "question": RunnablePassthrough(),  # 질문을 그대로 전달합니다.
} | RunnableLambda(map_docs)  # 사용자 정의 함수(map_docs)를 통해 문서를 처리 / RunnableLambda는 여기서 map_docs 호출

# **최종 프롬프트 정의**
# 추출된 문서와 질문을 사용하여 최종 답변을 생성하기 위한 프롬프트
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """ 
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context} 
            """,
        ),
        ("human", "{question}"),  # 질문 삽입
    ]
)

# **최종 체인 정의**
chain = {
    "context": map_chain,  # 문서 검색 및 관련 텍스트 추출
    "question": RunnablePassthrough(),  # 질문 그대로 전달
} | final_prompt | llm  # 최종 프롬프트와 언어 모델 연결

# **체인을 실행하여 질문에 답변 생성**
response = chain.invoke("How many ministries are mentioned")  # 질문 전달
print(response)  # 결과 출력


content='Three ministries are mentioned.'
